# Add environmental data
This notebook is the code to add environmental data to the output of pypam
For more information about this process please contact clea.parcerisas@vliz.be or check the documentation of both packages
https://lifewatch-pypam.readthedocs.io/en/latest/
https://github.com/lifewatch/bpnsdata

In [1]:
# Install the required packages. Geopandas can give problems in Windows machines, so better to install them using wheels when using Windows
import sys
!{sys.executable} -m pip install shapely==1.8.2
!{sys.executable} -m pip install bpnsdata
!{sys.executable} -m pip install pygeos

  Using cached Shapely-1.8.2-cp39-cp39-win_amd64.whl (1.3 MB)
  Attempting uninstall: shapely
    Found existing installation: Shapely 1.8.4
    Uninstalling Shapely-1.8.4:
      Successfully uninstalled Shapely-1.8.4


In [2]:
import os
import pathlib
import os

import geopandas
import numpy as np
import pandas as pd
import xarray
from tqdm import tqdm

import bpnsdata

C:\Users\cleap\Envs\sound-data-utils39\lib\site-packages\geopandas\_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(


In [3]:
raw_data_path = pathlib.Path('./data/raw_data')
processed_data_path = pathlib.Path('./data/processed')
metadata_path = raw_data_path.joinpath('data_summary_mda.csv')
metadata = pd.read_csv(metadata_path)

if not processed_data_path.exists():
    os.mkdir(processed_data_path)

# survey_params
binsize = 1.0
n_join_bins = 5
join_bins_overlap = 0.6

env_vars = [
		"shipping",
		"time",
		"habitat_suitability",
		"seabed_habitat",
		"sea_surface",
		"sea_wave",
		"wrakken_bank",
		"bathymetry"
    ]

In [4]:
def group_ds(ds, binsize, n_join_bins=None, join_bins_overlap=None):
    if n_join_bins not in [1, None]:
        n_overlap = (1 - join_bins_overlap) * n_join_bins
        if not n_overlap.is_integer():
            print('Warning, the overlap percentage of bins is not an integer. It will be set to the closer integer')
        n_overlap = int(n_overlap)
        time_window = list((np.arange(0, n_join_bins)) * binsize)
        grouped_id = 0
        new_ds = xarray.Dataset()

        for filename, file_ds in ds.groupby('file_path'):
            start_groups_id = np.arange(file_ds.id.min(), file_ds.id.max(), n_overlap)
            print('Grouping file %s' % filename)
            for start_id_small_window in tqdm(start_groups_id, total=len(start_groups_id - 1), position=0, leave=True):
                # Only add the windows that are complete!
                if start_id_small_window + n_join_bins < file_ds.id.max():
                    selected_ids = np.arange(start_id_small_window, start_id_small_window + n_join_bins)
                    small_window = file_ds.sel(id=selected_ids)
                    small_window = small_window.expand_dims('grouped_id')
                    small_window = small_window.assign_coords({'time_window': ('id', time_window[:len(selected_ids)]),
                                                               'grouped_id': [grouped_id]})
                    small_window = small_window.swap_dims({'id': 'time_window'})

                    if grouped_id == 0:
                        new_ds = small_window
                    else:
                        new_ds = xarray.concat((new_ds, small_window), 'grouped_id')
                    grouped_id += 1
        new_ds = new_ds.assign_coords({'grouped_datetime': ('grouped_id', new_ds.sel(time_window=0.0).datetime.values),
                                       'grouped_start_sample': ('grouped_id',
                                                                new_ds.sel(time_window=0.0).start_sample.values),
                                       'grouped_end_sample': ('grouped_id',
                                                              new_ds.sel(time_window=time_window[-1]).end_sample.values)
                                       })
        ds = new_ds
    return ds

In [ ]:
# Define the seadatamanager
manager = bpnsdata.SeaDataManager(env_vars)
id_name = 'grouped_id'
datetime_name = 'grouped_datetime'
for i, row in tqdm(metadata.iterrows(), total=len(metadata)):
    deployment_path = raw_data_path.joinpath('deployments', row['deployment_path'])
    env_path = processed_data_path.joinpath(row['deployment_path'].replace('.nc', '_env.nc'))
    print(env_path)
    if not env_path.exists():
        gps_path = raw_data_path.joinpath('gps', row['gps_path'])

        # Read the dataset
        ds_deployment = xarray.open_dataset(deployment_path)
        ds_deployment = group_ds(ds_deployment, binsize=binsize,
                                 n_join_bins=n_join_bins, join_bins_overlap=join_bins_overlap)
        # Get the time information from the dataset to get a pandas df
        datetime_index = ds_deployment[datetime_name].to_index()
        df = pd.DataFrame({"datetime": datetime_index.values, 'id': ds_deployment[id_name]})
        df = df.drop_duplicates("datetime")
        print(metadata.iloc[i]['deployment_name'], len(datetime_index), len(df))
        df = df.set_index('datetime')
        df.index = df.index.tz_localize('UTC')

        # Generate the location information
        geodf = manager.add_geodata(df, gps_path, time_tolerance='5s')
        geodf = manager.survey_location.add_distance_to_coast(geodf, coastfile='./geo/belgium_coast/basislijn_BE.shp')
        geodf_env = manager(geodf)

        # Remove the UTC (xarray does not support it?)
        geodf_env.index = geodf_env.index.tz_localize(None)
        lat = geodf_env['geometry'].y
        lon = geodf_env['geometry'].x
        df_env = geodf_env.drop(columns=['geometry', 'id'])
        env_ds = df_env.to_xarray()
        env_ds = env_ds.assign_coords(coords={'lat': lat, 'lon': lon, id_name : ('datetime', df.id.values)})
        env_ds = env_ds.swap_dims({'datetime': id_name})

        # Clean the previous if not all computed
        if len(env_ds[id_name]) != len(ds_deployment[id_name]):
            env_ds = env_ds.reindex_like(ds_deployment)
        new_ds = ds_deployment.merge(env_ds, compat="override")
        new_ds['season'] = new_ds[datetime_name].dt.isocalendar().week

        encoding = {'file_path': {'dtype': 'unicode'},
                    'start_sample': {'dtype': int, '_FillValue': -1},
                    'end_sample': {'dtype': int, '_FillValue': -1},
                    'datetime': {'dtype': float, '_FillValue': -1}}
        new_ds.to_netcdf(env_path,  encoding=encoding)

  0%|          | 0/67 [00:00<?, ?it/s]

data\processed\0_Heinkel 111_env.nc
Grouping file \\qarchive\data_sensors\b&k\COVID-19\20200429_Zeekat\Heinkel 111\200429095249_810_mono1.wav


100%|██████████| 1532/1532 [00:16<00:00, 92.72it/s]


Heinkel 111 1529 1529
Adding shipping...



100%|██████████| 1/1 [00:00<00:00,  2.63it/s]

...

  0%|          | 0/574 [00:00<?, ?it/s]

Adding habitat_suitability...
Adding seabed_habitat...
Adding sea_surface...
Adding sea_wave...
Adding wrakken_bank...
Adding bathymetry...


 42%|████▏     | 28/67 [11:18<16:38, 25.59s/it]

data\processed\28_Grafton_env.nc
Grouping file \\qarchive\data_sensors\b&k\COVID-19\20200713_Simon Stevin\Grafton_31.6\200713121943_889_mono1.wav


 12%|█▏        | 33/279 [00:00<00:02, 99.73it/s] 